# MLCourse.ai Kaggle Competition In Class 1

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve

## Naive Baseline

Objective: Rapidly make a training and testing pipeline. Will only develop functions to clean/standardize data, train a model and return test results.

In [7]:
!ls -lsa ../data

total 237824
     0 drwxr-xr-x  8 ns  staff       256  9 Apr 23:33 .
     0 drwxr-xr-x  4 ns  staff       128  9 Apr 23:23 ..
 53248 -rw-r--r--  1 ns  staff  26743371  9 Apr 23:29 catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2.zip
  2408 -rwxr-xr-x  1 ns  staff   1230867 28 Nov 13:13 sample_submission.csv
  3520 -rwxr-xr-x  1 ns  staff   1800682 28 Nov 13:13 site_dic.pkl
 37992 -rwxr-xr-x  1 ns  staff  19450729 28 Nov 13:13 test_sessions.csv
 22432 -rwxr-xr-x  1 ns  staff  11483103 28 Nov 13:13 train.zip
118224 -rwxr-xr-x  1 ns  staff  60526801 28 Nov 13:14 train_sessions.csv


In [47]:
#read in training data
data = pd.read_csv('../data/train_sessions.csv')
data.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [11]:
#read in site dictionary
sites = pd.Series(pd.read_pickle('../data/site_dic.pkl'))
sites.head()

www.abmecatronique.com                25075
groups.live.com                       13997
majeureliguefootball.wordpress.com    42436
cdt46.media.tourinsoft.eu             30911
www.hdwallpapers.eu                    8104
dtype: int64

In [14]:
#Are sites unique?
print(f'Names of sites in the set and the unique sites is equal? {len(sites.index.unique()) == len(sites)}')

Names of sites in the set and the unique sites is equal? True


In [15]:
#investigate data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253561 entries, 0 to 253560
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  253561 non-null  int64  
 1   site1       253561 non-null  int64  
 2   time1       253561 non-null  object 
 3   site2       250098 non-null  float64
 4   time2       250098 non-null  object 
 5   site3       246919 non-null  float64
 6   time3       246919 non-null  object 
 7   site4       244321 non-null  float64
 8   time4       244321 non-null  object 
 9   site5       241829 non-null  float64
 10  time5       241829 non-null  object 
 11  site6       239495 non-null  float64
 12  time6       239495 non-null  object 
 13  site7       237297 non-null  float64
 14  time7       237297 non-null  object 
 15  site8       235224 non-null  float64
 16  time8       235224 non-null  object 
 17  site9       233084 non-null  float64
 18  time9       233084 non-null  object 
 19  si

In [19]:
#pct nan in the dataset
data.isnull().sum()/len(data)

session_id    0.000000
site1         0.000000
time1         0.000000
site2         0.013657
time2         0.013657
site3         0.026195
time3         0.026195
site4         0.036441
time4         0.036441
site5         0.046269
time5         0.046269
site6         0.055474
time6         0.055474
site7         0.064142
time7         0.064142
site8         0.072318
time8         0.072318
site9         0.080758
time9         0.080758
site10        0.088772
time10        0.088772
target        0.000000
dtype: float64

In [46]:
def clean_data(data):
    data = data.fillna(0)
    return data

def create_features():
    pass
    
def split_data(data, cols=None):
    
    X = data[cols].astype('int')
    Y = data['target']
    train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.15, random_state=42)
    
    return train_x, test_x, train_y, test_y


def train_model(x, y, C=0.7, solver='lbfgs'):
    clf = LogisticRegression(C=C, solver=solver, random_state=42, n_jobs=4).fit(x, y)
    return clf

def evaluate_model(model, x, y_true):
    result = model.predict(x)
    return roc_auc_score(y_true, result)
    

df = clean_data(data)
    
sites = [f'site{i}' for i in range(1,11)]

train_x, test_x, train_y, test_y = split_data(df, cols=sites)

model = train_model(train_x, train_y, C=0.9)

roc_train = evaluate_model(model, train_x, train_y)
roc_test = evaluate_model(model, test_x, test_y)


print(f'Roc Auc Train: {roc_train}')
print(f'Roc Auc Test: {roc_test}')

Roc Auc Train: 0.5
Roc Auc Test: 0.5


### Naive Baseline Summary

What have we achieved? 
- In terms of results the predictions are random. The model is unable to learn anything. The question now becomes  *what insights can we derive from the data to learn from* because on a naive baiss the model does not learn.
- We have a simple pipeline to run and evaluate the model. GOOD! This makes us quicker to get through iterating ideas.

What have we ignored?
- We assumed that the site information held some feature that correlated to the target. We have no attepted to create a feature that correlates with this.
- We have ignored balancing the dataset for target variables.
- We have ignored the information encoded in the timestamps

## Add Site Frequency Feature

One assumption we can make is that the more often a site is visited the more likely we can identify the visitor. We can use countvectorizer for this. 

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

In [63]:
#try count vectorizer on the site integer data cast as a string
df = clean_data(data[sites])
df = df.astype('str')
X = CountVectorizer().fit_transform(df)
X.toarray()

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [66]:
#the result above is wrong. We should see multiple counts per site. 
#Therefore we need a different method to get the counts.
df['site1'].value_counts()

21       13874
23        9244
782       7769
29        5501
22        5434
         ...  
33159        1
34619        1
24379        1
21552        1
21734        1
Name: site1, Length: 15765, dtype: int64